<a href="https://colab.research.google.com/github/IvanRobotics/Autonomous-Follow-Me-Cooler/blob/master/AIBOY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Libraries </h3>

*   **openai**: This library provides an API to access the OpenAI platform, which offers a range of artificial intelligence services, such as natural language processing, image recognition, and more. With this library, developers can easily integrate OpenAI's services into their applications and leverage the power of AI to enhance their functionality.
*   **os**: This library provides a way to interact with the operating system, such as creating, deleting, and manipulating files and directories, managing processes, and more. It is essential for many applications that need to interact with the file system or the system itself.
*   **gradio**: This library provides a simple interface to build and deploy customizable UI components for machine learning models. It is useful for building interactive demos, sharing models with non-technical users, and rapidly prototyping model architectures.
*  **spacy**: This library provides industrial-strength natural language processing tools for Python. It includes functionality for tokenization, POS tagging, named entity recognition, and more. It is widely used in applications that require sophisticated text analysis and processing.
* **knowledge**: It contains Knowledge bases which are useful for applications such as question answering, chatbots, and expert systems.
* **components**: This is a sub-module of the gradio library and provides pre-built UI components that can be used to build custom interfaces for machine learning models. It includes components for image input/output, text input/output, sliders, and more.



In [ ]:
import openai
import os
import gradio as gr
import spacy
from knowledge import knowledge_base
from gradio import components

* **openai.api_key = os.environ.get('OPENAI_API_KEY')**: This line sets the API key for OpenAI's services, which is retrieved from an environment variable. This is a best practice for security reasons, as it keeps the API key hidden from public view.

* **nlp = spacy.load("en_core_web_lg")**: This line loads the "en_core_web_lg" model from the spaCy library, which is a large English language model that includes word vectors, syntax, and named entity recognition. This is a common model used in natural language processing applications.

* **messages = [{"role": "system", "content": "You are a helpful and kind AI Assistant."},]**: This creates a list of message objects, where each message has a "role" and "content" attribute. In this case, there is only one message, which is a system message that compliments the AI assistant. This list of messages is likely used in some kind of conversational AI application.

In [ ]:
openai.api_key = os.environ.get('OPENAI_API_KEY')
nlp = spacy.load("en_core_web_lg")
custom_css = "./style.css"

messages = [
    {"role": "system", "content": "You are a helpful and kind AI Assistant."},
]





**get_matching_question function**


* This function searches through a knowledge base of questions and answers, and returns the matching question and its corresponding answer if the input contains a question from the knowledge base. If there is no match,the function returns None for both question and answer.

In [ ]:
def get_matching_question(input):
    """
    Returns the matching question from the knowledge base, if any.
    """
    for question, answer in knowledge_base.items():
        if question in input.lower():
            return question, answer
    return None, None

**get_most_similar_question**

* This function finds the most similar question in the knowledge base to the input text using the spaCy library. It returns the most similar question if one is found, or None if there are no similar questions in the knowledge base.

In [ ]:
def get_most_similar_question(input):
    """
    Returns the most similar question from the knowledge base using Spacy.
    """
    similarities = []
    for question in knowledge_base.keys():
        try:
            similarity = nlp(question).similarity(nlp(input))
            similarities.append((question, similarity))
        except:
            pass

    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    most_similar_question = similarities[0][0] if similarities else None
    return most_similar_question

**check_sentiment**

* This functions check the sentiment of the input sentences, and the AI will generate the reponses correspondingly 

In [ ]:
def check_sentiment(input):
    """
    Checks the sentiment of the input using Spacy's text classification model.
    """
    doc = nlp(input)
    return doc.cats["POSITIVE"] > doc.cats["NEGATIVE"]

**chatbot**

* This function processes input text for a conversational AI application. It adds the input to the list of messages, checks if the input matches a question in the knowledge base, and if not, finds the most similar question using Spacy.If there is no matching question, it checks the sentiment of the input and responds accordingly.

In [ ]:

def chatbot(input):
    if input:
        messages.append({"role": "user", "content": input})

        # check if input matches any question in the knowledge base
        question, answer = get_matching_question(input)
        if question:
            messages.append({"role": "assistant", "content": answer})
            return answer

        # if no matching question is found, use Spacy to find the most similar question in the knowledge base
        most_similar_question = get_most_similar_question(input)
        if most_similar_question:
            answer = knowledge_base[most_similar_question]
            messages.append({"role": "assistant", "content": answer})
            return answer

        # check the sentiment of the input
        is_positive = check_sentiment(input)

        if is_positive:
            messages.append({"role": "assistant", "content": "I'm glad to hear that!"})
            return "I'm glad to hear that!"
        else:
            messages.append({"role": "assistant", "content": "I'm sorry to hear that."})
            return "I'm sorry to hear that."


<h3>
Set up Gradio Interface 
</h3>


In [ ]:
inputs = components.Textbox(lines=7, label="Chat with AI BOY")
outputs = components.Textbox(label="Reply")
gr.Interface(
    fn=chatbot,
    inputs=inputs,
    outputs=outputs,
    title="Welcome to My AI Chatbot",
    description="Feel free to ask me anything about Ivan's resume!",
    css=custom_css,
).launch()